★実行環境の選択

In [ ]:
# VSCodeの場合
edi_flg = 1
# Googlre Colabの場合
# edi_flg = 0

In [ ]:
if edi_flg == 0:
  # GoogleColabにGitHubリポジトリをクローンする用
  %cd /content/My_Python_project

  from google.colab import auth
  auth.authenticate_user()
  !git clone https://github.com/keiseki-eng/My_Python_project
  !git pull origin main



In [ ]:
# import sys
# !{sys.executable} -m pip install ipykernel

In [ ]:
if edi_flg == 0:
    !pip install japanize-matplotlib

import os
import sys
import yaml
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import japanize_matplotlib #日本語表示対応
from sklearn.model_selection import KFold


# Notebook から src ディレクトリを追加
# sys.path.append(os.path.join(os.getcwd(), "src"))
sys.path.append("/home/keiseki/My_Python_project/src")


In [ ]:
if edi_flg == 0:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=pd.errors.PerformanceWarning
)

## 01.config読み込み

In [ ]:
# VSCode用
if edi_flg==1:
  conf_path = os.path.join( '../config/config.yaml')
  with open(conf_path, 'r') as f:
    config = yaml.safe_load(f)

# GoogleColab用
elif edi_flg==0:
  conf_path = "My_Python_project/config/config.yaml"
  with open(conf_path, "r") as f:
    config = yaml.safe_load(f)

In [ ]:
# 定義した特徴量リストを読み込み
feature_list = config['FEATURE']['FEATURE_LIST']

In [ ]:
# カテゴリカルカラムのリストを定義
categorical_cols = config['FEATURE']['CATEGORICAL_COLS']

## 02.データ読み込み

In [ ]:
# VSCode用
if edi_flg==1:
# ドライブ内のファイルパスを指定
    train_path = '../20.Data/processed_train.pkl'
    df_train = pd.read_pickle(train_path)

# GoogleColab用
elif edi_flg==0:
    # GoogleDriveをマウントしてファイル読み込み準備
    from google.colab import drive
    drive.mount('/content/drive')
    # ドライブ内のファイルパスを指定
    train_path = '/content/drive/MyDrive/Colab Notebooks/不動産予測/processed_train.pkl'
    df_train = pd.read_pickle(train_path)

In [ ]:
# testデータの読み込み
# VSCode用
if edi_flg==1:
    test_path = '../20.Data/processed_test.pkl'
    df_test = pd.read_pickle(test_path)

# GoogleColab用
elif edi_flg==0:
    # ドライブ内のファイルパスを指定
    test_path = '/content/drive/MyDrive/Colab Notebooks/不動産予測/processed_test.pkl'
    df_test = pd.read_pickle(test_path)

In [ ]:
col_list = [
    
    "reform_exterior_date", 
    'total_floor_area',
    'building_area',
    'building_land_area',
    'land_area_all',
    'room_count',
    'unit_area',
    'reform_date',
    'reform_wet_area_date',
    'reform_interior_date',
    'bus_time1',
    'walk_distance1',
    'bus_time2',
    'walk_distance2',
    'madori_number_all',
    'money_rimawari_now',
    'parking_distance',
    'school_ele_distance',
    'school_jun_distance',
    'convenience_distance',
    'super_distance',
    'hospital_distance',
    'park_distance',
    'drugstore_distance',
    'bank_distance',

    'building_age',
    'years_since_renovation',
]

In [ ]:
# 指定したカラムについて、カラムごとのパーセンタイル３分割でカテゴリ変数化
def categorize_by_percentile_3cols(df, cols):
    for col in cols:
        p1, p2 = np.percentile(df[col], [33.33, 66.67])

        if p1 == p2:
            # 2分割にフォールバック
            df[col + "_flag"] = pd.cut(
                df[col],
                bins=[-np.inf, p1, np.inf],
                labels=[0, 1]
            ).astype(int)
        else:
            df[col + "_flag"] = pd.cut(
                df[col],
                bins=[-np.inf, p1, p2, np.inf],
                labels=[0, 1, 2]
            ).astype(int)


In [ ]:
categorize_by_percentile_3cols(df_train, col_list)
categorize_by_percentile_3cols(df_test, col_list)

In [ ]:
# 作成した特徴量を特徴量リストとカテゴリ特徴量リストに追加
df_train.columns.to_list()

In [ ]:
# 作成したデータセットを新しい特徴量ファイルとして保存
# VSCode用
if edi_flg==1:
    train_path = '../20.Data/processed_train_with_flags.pkl'
    df_train.to_pickle(train_path)
    train_path = '../20.Data/processed_test_with_flags.pkl'
    df_train.to_pickle(train_path)